In [254]:
import pandas as pd
import numpy as np
import re

fires = pd.read_csv("./2024USFires.csv")

In [255]:
fires.head()

,Name,State,County,Acres,Start Date,Containment Date
0,National,Alaska,Yukon-Koyukuk,"15,381",June 30,October 3
1,Sushana River,Alaska,Denali,"1,261",June 27,October 9
2,Quinn Creek,Alaska,Bethel,"2,709",June 26,August 16
3,East Toklat,Alaska,Denali,"29,612",June 26,October 9
4,Gold King Creek,Alaska,Denali,"6,538",June 26,October 9


In [256]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              472 non-null    object
 1   State             472 non-null    object
 2   County            472 non-null    object
 3   Acres             472 non-null    object
 4   Start Date        472 non-null    object
 5   Containment Date  471 non-null    object
dtypes: object(6)
memory usage: 22.3+ KB


In [257]:
# Add the year as a column
fires['Year'] = 2024

In [258]:
# Replace spaces in counties with ''
fires['County'] = fires['County'].apply(lambda x: x.replace(' ', ''))

# Concatenate State and County as State_County
fires['State_County'] = fires['State'] + '_' + fires['County']

In [259]:
# Change start and containment date column values to only include month
fires['Start'] = fires['Start Date'].str.replace(r' \d*', '', regex=True)
fires['Start'] = fires['Start'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

fires['End'] = fires['Containment Date'].str.replace(r' \d*', '', regex=True)
fires['End'] = fires['End'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

# Only select desired columns
    # Note: County should NOT be used since county names may be repeated across states and could compromise data analysis
fires = fires[['State', 'State_County', 'Start', 'End']]

fires.head()

,State,State_County,Start,End
0,Alaska,Alaska_Yukon-Koyukuk,June,October
1,Alaska,Alaska_Denali,June,October
2,Alaska,Alaska_Bethel,June,August
3,Alaska,Alaska_Denali,June,October
4,Alaska,Alaska_Denali,June,October


In [260]:
# Print set of all values in the Start and End columns
months = set(fires.loc[:,'Start']).union(set(fires.loc[:,'Start']))
print(months)

{'July', 'August', 'September', 'May', 'February', 'November', 'March', 'June', 'October', 'April', 'December'}


In [261]:
# Convert months to digits and add the range of months the fire lasted
replace_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12, 'unknown':  np.nan}
fires.Start = fires.Start.map(replace_map)
fires.End = fires.End.map(replace_map)

In [262]:
fires['Start'].astype(float)
fires['End'].astype(float)

0      10.0
1      10.0
2       8.0
3      10.0
4      10.0
       ... 
467     9.0
468     9.0
469     9.0
470     9.0
471     9.0
Name: End, Length: 472, dtype: float64

In [263]:
fires.head()

,State,State_County,Start,End
0,Alaska,Alaska_Yukon-Koyukuk,6.0,10.0
1,Alaska,Alaska_Denali,6.0,10.0
2,Alaska,Alaska_Bethel,6.0,8.0
3,Alaska,Alaska_Denali,6.0,10.0
4,Alaska,Alaska_Denali,6.0,10.0


In [264]:
# How many NaN's? Is it safe to remove?
fires.isna().sum()

# Take out the fires that we don't know when they endedd
fires = fires.dropna()

In [265]:
# Which months did each fire burn for? All numbers between start and end, inclusive on both ends.

fires['Months'] = [[] for _ in range(len(fires))]

for idx in fires.index:
    start = int(fires.loc[idx, 'Start'])
    end = int(fires.loc[idx, 'End'])

    
    months = list(range(start, end + 1))
    
    fires.at[idx, 'Months'] = months

    print(idx, fires.loc[idx, 'Months'])


0 [6, 7, 8, 9, 10]
1 [6, 7, 8, 9, 10]
2 [6, 7, 8]
3 [6, 7, 8, 9, 10]
4 [6, 7, 8, 9, 10]
5 [6, 7, 8, 9, 10]
6 [6, 7, 8, 9, 10]
7 [6, 7, 8, 9, 10]
8 [6, 7, 8, 9, 10]
9 [6, 7, 8, 9]
10 [6, 7, 8]
11 [6, 7, 8, 9, 10]
12 [6, 7, 8, 9, 10]
13 [6, 7, 8, 9]
14 [6, 7, 8, 9]
15 [6, 7, 8, 9, 10]
16 [6, 7, 8, 9, 10]
17 [6, 7, 8, 9]
18 [6, 7, 8]
19 [6, 7, 8]
20 [6, 7, 8]
21 [6, 7, 8]
22 [6, 7, 8]
23 [6, 7, 8]
24 [6, 7, 8]
25 [6, 7, 8]
26 [6, 7]
27 [6, 7, 8, 9]
28 [6, 7, 8, 9]
29 [9]
30 [5, 6, 7]
31 [5]
32 [5, 6, 7]
33 [5]
35 [3]
36 [6, 7, 8]
37 [6, 7]
38 [6, 7]
39 [6, 7, 8, 9]
40 [6]
41 [6, 7]
42 [6]
43 [7, 8]
44 [7, 8]
45 [7, 8]
46 [7, 8]
47 [7, 8]
48 [7, 8]
49 [7, 8]
50 [7, 8]
51 [7, 8, 9]
52 [7, 8]
53 [7, 8]
54 [7, 8]
55 [7]
56 [8, 9]
57 [8, 9]
58 [8]
59 [8]
60 [8]
61 [8, 9]
62 [8]
63 [4, 5, 6, 7]
64 []
65 [10]
66 [10]
67 [9, 10]
68 [9]
69 [9]
71 [8, 9]
72 [7]
73 [7]
74 [7]
76 [6]
77 [6]
78 [6]
79 [7, 8, 9]
80 [12]
81 [11]
82 [10, 11, 12]
83 [10]
84 [10, 11]
85 [9, 10]
86 [9, 10, 11]
87 [9]
88 [9,

In [266]:
print(fires)

       State          State_County  Start   End            Months
0     Alaska  Alaska_Yukon-Koyukuk    6.0  10.0  [6, 7, 8, 9, 10]
1     Alaska         Alaska_Denali    6.0  10.0  [6, 7, 8, 9, 10]
2     Alaska         Alaska_Bethel    6.0   8.0         [6, 7, 8]
3     Alaska         Alaska_Denali    6.0  10.0  [6, 7, 8, 9, 10]
4     Alaska         Alaska_Denali    6.0  10.0  [6, 7, 8, 9, 10]
..       ...                   ...    ...   ...               ...
467  Wyoming      Wyoming_Converse    9.0   9.0               [9]
468  Wyoming      Wyoming_Campbell    9.0   9.0               [9]
469  Wyoming       Wyoming_Natrona    9.0   9.0               [9]
470  Wyoming    Wyoming_HotSprings    9.0   9.0               [9]
471  Wyoming        Wyoming_Weston    8.0   9.0            [8, 9]

[453 rows x 5 columns]


In [267]:
# Replace map to change numbers to month names for readability
replace_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December', np.nan: 'unknown'}

fires.Start = fires.Start.map(replace_map)
fires.End = fires.End.map(replace_map)

fires.head()

,State,State_County,Start,End,Months
0,Alaska,Alaska_Yukon-Koyukuk,June,October,"[6, 7, 8, 9, 10]"
1,Alaska,Alaska_Denali,June,October,"[6, 7, 8, 9, 10]"
2,Alaska,Alaska_Bethel,June,August,"[6, 7, 8]"
3,Alaska,Alaska_Denali,June,October,"[6, 7, 8, 9, 10]"
4,Alaska,Alaska_Denali,June,October,"[6, 7, 8, 9, 10]"


In [268]:
# Explode the months column for ease of replace map usage
exploded_fires = fires.explode('Months')

# Replace numbers with months once again
exploded_fires.Months = exploded_fires.Months.map(replace_map)

exploded_fires.head()

,State,State_County,Start,End,Months
0,Alaska,Alaska_Yukon-Koyukuk,June,October,June
0,Alaska,Alaska_Yukon-Koyukuk,June,October,July
0,Alaska,Alaska_Yukon-Koyukuk,June,October,August
0,Alaska,Alaska_Yukon-Koyukuk,June,October,September
0,Alaska,Alaska_Yukon-Koyukuk,June,October,October


In [269]:
# Replace fires with the exploded_fires df aggregated and grouped to recover original format (list in the Months column)
# Group by the columns you want to remain as unique IDs
# .agg(list) applies list agg function to each group, recreating the list
# Don't forget to reset index
fires = exploded_fires.groupby(['State', 'State_County', 'Start', 'End']).agg(list).reset_index()

# Convert the list to a string for readability
fires['Months'] = fires['Months'].apply(lambda x: ', '.join(x))

fires.head()

,State,State_County,Start,End,Months
0,Alaska,Alaska_Bethel,June,August,"June, July, August, June, July, August, June, ..."
1,Alaska,Alaska_Denali,June,October,"June, July, August, September, October, June, ..."
2,Alaska,Alaska_FairbanksNorthStar,June,October,"June, July, August, September, October, June, ..."
3,Alaska,Alaska_FairbanksNorthStar,June,September,"June, July, August, September, June, July, Aug..."
4,Alaska,Alaska_LakeandPeninsula,June,July,"June, July"


In [270]:
# Only keep one item in list if start and end month are the same
for idx in range(len(fires)):
    if fires.loc[idx, 'Start'] == fires.loc[idx, 'End']:
        fires.loc[idx, 'Months'] = [fires.loc[idx, 'Start']]

fires

,State,State_County,Start,End,Months
0,Alaska,Alaska_Bethel,June,August,"June, July, August, June, July, August, June, ..."
1,Alaska,Alaska_Denali,June,October,"June, July, August, September, October, June, ..."
2,Alaska,Alaska_FairbanksNorthStar,June,October,"June, July, August, September, October, June, ..."
3,Alaska,Alaska_FairbanksNorthStar,June,September,"June, July, August, September, June, July, Aug..."
4,Alaska,Alaska_LakeandPeninsula,June,July,"June, July"
...,...,...,...,...,...
366,Wyoming,"Wyoming_Platte,Goshen",July,August,"July, August"
367,Wyoming,Wyoming_Sheridan,September,November,"September, October, November"
368,Wyoming,"Wyoming_Sheridan,Campbell,Johnson",August,August,[August]
369,Wyoming,Wyoming_Teton&Fremont,September,November,"September, October, November"


In [271]:
# Create a column counting the number of months a fire burned, leave each entry empty for now (pd.NA so count can be an int)
fires['Num_Months'] = pd.NA

# For each row in fires df
for idx in range(len(fires)):
    # Initialize count variable as an int
    count = 0
    # For each item in the Months list
    for item in fires.loc[idx, 'Months']:
        count += 1
    fires.loc[idx, 'Num_Months'] = int(count)

fires.head()

,State,State_County,Start,End,Months,Num_Months
0,Alaska,Alaska_Bethel,June,August,"June, July, August, June, July, August, June, ...",138
1,Alaska,Alaska_Denali,June,October,"June, July, August, September, October, June, ...",198
2,Alaska,Alaska_FairbanksNorthStar,June,October,"June, July, August, September, October, June, ...",118
3,Alaska,Alaska_FairbanksNorthStar,June,September,"June, July, August, September, June, July, Aug...",60
4,Alaska,Alaska_LakeandPeninsula,June,July,"June, July",10


In [272]:
# Convert fires df to a cleaned csv, saved in user's cwd 
fires.to_csv('./2024USFires_cleaned.csv')